This code is meant to build and plot a dataframe which shows the % of total population enrolled in public schools for high growth counties in NC as part of the NC Growth Project 
m.hargroder@outlook.com

In [9]:
%matplotlib notebook
%matplotlib inline

# Dependencies 
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import time


Import raw files and build datafram with public shcool enrollment for years in analysis

In [7]:
enrol = pd.read_csv('data_import/cnty_PubSchoolEnroll12-18.csv')
#enrol.head()
 #drop AreaType, Var Name, and rename Value colum to PS_Enrollment
enrol.drop(columns=['Area Type','Variable'], inplace = True)
enrol.rename(columns ={'Value':'PS_Enrollment', 'Area Name':'County'}, inplace = True)
    
enrol.head()


,County,Year,PS_Enrollment
0,Wayne,2014,20318
1,Brunswick,2013,13629
2,Davidson,2013,25970
3,Hoke,2013,8510
4,Camden,2012,1958


In [22]:
pop = pd.read_csv('data_import/cnty_Pop12-18.csv')
pop.rename(columns ={'Area Name':'County'}, inplace = True)
pop.head()


,County,Year,Population
0,Duplin,2018,59772
1,Warren,2017,20249
2,Washington,2018,12153
3,Watauga,2016,55142
4,Wayne,2017,124086


Merge county enrollment and population dataframes 

In [29]:
df1 = pd.merge(enrol, pop , on = ['County','Year'], how = 'left')
#check shape, should still be 700 rows
df1.shape
#df1.head(10)
#clean up merge
df1.drop(columns=['Population_x'], inplace = True)
df1.rename(columns ={'Population_y':'Population'}, inplace = True)
df1.head(10)

,County,Year,PS_Enrollment,Population
0,Wayne,2014,20318,125646
1,Brunswick,2013,13629,115284
2,Davidson,2013,25970,163498
3,Hoke,2013,8510,50581
4,Camden,2012,1958,9966
5,Tyrrell,2013,576,4138
6,Bladen,2018,4933,34566
7,Gaston,2017,35076,218445
8,Montgomery,2018,3976,27621
9,Martin,2018,3529,23264


Import and high growth county file to to create a series and list so can identify high growth counties in df1

In [82]:
high_grwth_cnty = pd.read_csv('data_import/cnty_growth.csv', usecols= [0])
high_grwth_cnty['CO_NAME'] =high_grwth_cnty['CO_NAME'].str.title()
high_grwth_cnty.rename(columns ={'CO_NAME':'County'}, inplace = True)
grwth_Counties = high_grwth_cnty['County'].tolist()
grwth_Counties 

['Brunswick',
 'Pender',
 'Johnston',
 'Cabarrus',
 'Chatham',
 'Wake',
 'Mecklenburg',
 'Currituck',
 'Union',
 'New Hanover',
 'Durham',
 'Franklin']

Add column to df1 that returns boolean if county in high_grwth_cnty series

In [85]:
df1['High_Growth'] = df1.County.map(lambda x: True if x in ['Wayne'] else False ) 
df1.head()


,County,Year,PS_Enrollment,Population,High_Growth,PercPop_PSEnrolled
0,Wayne,2014,20318,125646,False,16.170829
1,Brunswick,2013,13629,115284,False,11.822109
2,Davidson,2013,25970,163498,False,15.883986
3,Hoke,2013,8510,50581,False,16.824499
4,Camden,2012,1958,9966,False,19.646799


Add column wich represents the % of population enrolled in PS for each row (County and Year)

In [72]:
df1['PercPop_PSEnrolled']  = (df1['PS_Enrollment'] /df1['Population'])*100

df1.head()


,County,Year,PS_Enrollment,Population,High_Growth,PercPop_PSEnrolled
0,Wayne,2014,20318,125646,False,16.170829
1,Brunswick,2013,13629,115284,False,11.822109
2,Davidson,2013,25970,163498,False,15.883986
3,Hoke,2013,8510,50581,False,16.824499
4,Camden,2012,1958,9966,False,19.646799


In [63]:
#Create Pivot and Plot
xtab = pd.pivot_table(df1, index='High_Growth', columns = 'Year', values = 'PercPop_PSEnrolled')
xtab.head(15)
#plt = xtab.plot(figsize=(12,8), title = 'County Crime Rate by Year')
#plt.tight_layout()
#plt.show()

Year,2012,2013,2014,2015,2016,2017,2018
High_Growth,,,,,,,
False,14.938648,14.898172,14.929806,14.899142,14.793203,14.491634,14.382939
